In [1]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os

print("✅ All imports successful")

✅ All imports successful


In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

✅ API key loaded


In [3]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [4]:
@tool
def calculator(expression: str) -> str:
    """
    Calculate mathematical expressions.
    
    Args:
        expression: Math expression like "2 + 2" or "15 * 37"
    """
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search(query: str) -> str:
    """
    Search for information (simulated).
    
    Args:
        query: The search query
    """
    # Simulated search results
    knowledge = {
        "python": "Python is a high-level programming language created in 1991.",
        "langgraph": "LangGraph is a framework for building stateful multi-actor applications.",
        "react": "ReAct is an agent pattern that combines reasoning and acting."
    }
    
    for key, value in knowledge.items():
        if key in query.lower():
            return value
    
    return "No information found."

tools = [calculator, search]
print("✅ Tools created")

✅ Tools created


In [5]:
# Custom state for Plan-Execute
class PlanExecuteState(TypedDict):
    """State for plan-execute pattern."""
    input: str  # Original user query
    plan: list[str]  # List of steps
    current_step: int  # Which step we're on
    results: Annotated[list[str], operator.add]  # Results from each step
    final_output: str  # Final answer

print("✅ Plan-Execute state defined")

✅ Plan-Execute state defined


In [6]:
class CritiqueItem(TypedDict):
    clarity: int         
    completeness: int             
    accuracy: int 



# Custom state for Reflection
class ReflectionState(TypedDict):
    """State for reflection pattern."""
    task: str  # Original task
    draft: str  # Current draft
    critique: list[CritiqueItem]
    iterations: int  # Number of refinements
    final_output: str  # Final result

    
MAX_REFLECTIONS = 5 


print("✅ Reflection state defined")

✅ Reflection state defined


In [7]:
# Node 1: Generator
def generator(state: ReflectionState) -> dict:
    """Generate or refine based on critique."""
    if state["iterations"] == 0:
        # First generation
        prompt = f"""Create a response for this task:

Task: {state['task']}

Provide a clear, complete answer."""
        print("\n✍️ Generating initial draft...")
    else:
        # Refinement based on critique
        prompt = f"""Improve this draft based on the critique scores:

Task: {state['task']}

Current draft: {state['draft']}

Critique: {state['critique']}

Create an improved version."""
        print(f"\n✍️ Refining (iteration {state['iterations']})...")
    
    response = llm.invoke([HumanMessage(content=prompt)])
    print("✓ Draft created\n")
    
    return {"draft": response.content}

# Node 2: Critic
def critic(state: ReflectionState) -> dict:
    """Evaluate draft and provide critique."""
    prompt = f"""Evaluate this response and provide constructive critique ONLY AS A LIST of three scoring (clarity, completeness and accuracy) on a scale of 1 to 5 each. (e.g 1. clarity: 4, 2.completeness: 3, accuracy: 4):

Task: {state['task']}

Response: {state['draft']}

critique the response ONLY AS A LIST of three scoring (clarity, completeness and accuracy) on a scale of 1 to 5 each. (e.g 1. clarity: 4, 2.completeness: 3, accuracy: 4)
"""
    
#     Critique the response. What could be improved?
# If it's excellent, say "APPROVED: explanation".
# Otherwise, provide specific improvements needed.
    
    print("🔍 Critiquing draft...")
    response = llm.invoke([HumanMessage(content=prompt)])

    critique_list = []


    critique_response = response.content
    print(critique_response)

    
    for line in critique_response.strip().splitlines():
        _, rest = line.split(".", 1)          # remove numbering
        criterion, score = rest.split(":")    # split key/value
        # print(f"{criterion} : {score}")
        critique_list.append({
           criterion.strip(): int(score.strip())
        })

    print(critique_list)


    # for criticizm, value in critique_response.item():

    
    # print(f"Critique: {critique_response[:100]}\n")
    
    return {
        "critique": critique_list,
        "iterations": state["iterations"] + 1
    }

# Node 3: Finalizer
def reflection_finalizer(state: ReflectionState) -> dict:
    """Set final output."""
    print("\n✅ Reflection complete!\n")
    return {"final_output": state["draft"]}

print("✅ Reflection nodes defined")

✅ Reflection nodes defined


In [8]:
# Routing function
def should_reflect_again(state: ReflectionState) -> Literal["generator", "finalizer"]:
    """Decide if we need more refinement."""
    # Stop if approved or max iterations

    critique_scores = state.get("critique", [])
    scores = [score for item in critique_scores for score in item.values() ]
        

    if all(score >= 4 for score in scores):
        return "finalizer"
        
    if state["iterations"] >= MAX_REFLECTIONS:
        print(f"⚠️ Max iterations ({MAX_REFLECTIONS}) reached\n")
        return "finalizer"
    
    return "generator"

# Build Reflection graph
reflection_builder = StateGraph(ReflectionState)

reflection_builder.add_node("generator", generator)
reflection_builder.add_node("critic", critic)
reflection_builder.add_node("finalizer", reflection_finalizer)

reflection_builder.add_edge(START, "generator")
reflection_builder.add_edge("generator", "critic")
reflection_builder.add_conditional_edges(
    "critic",
    should_reflect_again,
    {"generator": "generator", "finalizer": "finalizer"}  # Can loop back
)
reflection_builder.add_edge("finalizer", END)

reflection_agent = reflection_builder.compile()

print("✅ Reflection agent created")

✅ Reflection agent created


In [9]:
result = reflection_agent.invoke({
    "task": "Explain what an agentic pattern is in 2-3 sentences",
    "draft": "",
    "critique": [],
    "iterations": 0
})

print(f"\n{'='*70}")
print("📊 FINAL OUTPUT (after reflection):")
print(f"{'='*70}")
print(result["final_output"])
print(f"\nTotal iterations: {result['iterations']}")
print(f"{'='*70}\n")


✍️ Generating initial draft...
✓ Draft created

🔍 Critiquing draft...
1. clarity: 5  
2. completeness: 4  
3. accuracy: 5  
[{'clarity': 5}, {'completeness': 4}, {'accuracy': 5}]

✅ Reflection complete!


📊 FINAL OUTPUT (after reflection):
An agentic pattern refers to a behavioral or cognitive framework in which individuals take active control over their actions and decisions, demonstrating autonomy and self-efficacy. This pattern emphasizes the capacity of individuals to influence their own circumstances and outcomes, rather than being passive recipients of external forces. In essence, it highlights the proactive role that people play in shaping their lives and environments.

Total iterations: 1



In [10]:
result = reflection_agent.invoke({
    "task": "Explain linear algebra",
    "draft": "",
    "critique": [],
    "iterations": 0
})

print(f"\n{'='*70}")
print("📊 FINAL OUTPUT (after reflection):")
print(f"{'='*70}")
print(result["final_output"])
print(f"\nTotal iterations: {result['iterations']}")
print(f"{'='*70}\n")


✍️ Generating initial draft...
✓ Draft created

🔍 Critiquing draft...
1. clarity: 5  
2. completeness: 5  
3. accuracy: 5  
[{'clarity': 5}, {'completeness': 5}, {'accuracy': 5}]

✅ Reflection complete!


📊 FINAL OUTPUT (after reflection):
Linear algebra is a branch of mathematics that deals with vectors, vector spaces, linear transformations, and systems of linear equations. It is fundamental in various fields, including engineering, physics, computer science, economics, and statistics, due to its ability to model and solve problems involving linear relationships.

### Key Concepts in Linear Algebra:

1. **Vectors**: 
   - A vector is an ordered list of numbers, which can represent points in space, directions, or quantities. For example, in two-dimensional space, a vector can be represented as \( \mathbf{v} = [v_1, v_2] \), where \( v_1 \) and \( v_2 \) are its components.

2. **Vector Spaces**: 
   - A vector space is a collection of vectors that can be added together and multiplied